In [ ]:
%cd ..

%config InlineBackend.figure_format = "retina"

/home/marco/thesis_project/contrast-gan-3D


In [ ]:
from pathlib import Path
from pprint import pprint

import numpy as np
import pandas as pd
from IPython.display import display
from matplotlib import pyplot as plt

from contrast_gan_3D import config as project
from contrast_gan_3D.constants import VMAX, VMIN
from contrast_gan_3D.data import utils as data_u
from contrast_gan_3D.eval import marker_recall_rate as mrr
from contrast_gan_3D.utils import geometry as geom
from contrast_gan_3D.utils import io_utils
from contrast_gan_3D.utils import visualization as viz

In [ ]:
annotations_root = project.ANNOTATIONS_DIR / "IDR_CADRADS"
annotation_fnames = sorted(list(annotations_root.glob("*")))
annotation_names = [io_utils.stem(el) for el in annotation_fnames]
print(len(annotation_names))
print(annotation_names[:3])
print(annotation_fnames[:3])

idr_root = project.DATA_DIR / "IDR_CADRADS"
idr_scans_fnames = sorted(
    [el for el in idr_root.glob("*.mhd") if io_utils.stem(el) in annotation_names]
)
print(len(idr_scans_fnames))
print(idr_scans_fnames[:3])

96
['3153', '3169', '3177']
[PosixPath('/home/marco/data/annotations/IDR_CADRADS/3153'), PosixPath('/home/marco/data/annotations/IDR_CADRADS/3169'), PosixPath('/home/marco/data/annotations/IDR_CADRADS/3177')]
96
[PosixPath('/home/marco/data/IDR_CADRADS/3153.mhd'), PosixPath('/home/marco/data/IDR_CADRADS/3169.mhd'), PosixPath('/home/marco/data/IDR_CADRADS/3177.mhd')]


In [ ]:
df = pd.read_excel(project.DATA_DIR / "ostia_test_IDR_CADRADS_label.xlsx")
labels = df.loc[df["ID"].isin(annotation_names), ["ID", "label"]]
display(labels)
display(labels["label"].value_counts())

,ID,label
6,3153,1
7,3169,1
9,3177,1
10,3188,1
11,3192,1
...,...,...
299,Contrast2794,1
301,Contrast2816,-1
304,Contrast2895,1
308,Contrast2952,1


label
 1    79
-1    17
Name: count, dtype: int64

In [ ]:
RUN_NAME = "3D-WC-small-rmsprop"

correction_fnames = list((project.CORRECTIONS_DIR / RUN_NAME).glob("*"))

rng = np.random.default_rng()
idx = rng.choice(len(correction_fnames))

if False:
    test_file = correction_fnames[idx]
    print(
        test_file,
        idx,
        labels.loc[labels["ID"] == io_utils.stem(test_file), "label"],
    )

    scan, meta = io_utils.load_sitk_image(test_file)
    print(scan.min(), scan.max())

    fig = viz.plot_mid_slice(scan)[0].get_figure()
    plt.show()
    plt.close(fig)

    slice_idxs = sorted(rng.choice(scan.shape[-1], size=64, replace=False))
    print(scan[None, ..., slice_idxs].shape)
    fig = viz.plot_axial_slices(
        scan[None, ..., slice_idxs], normalize=True, value_range=(VMIN, VMAX)
    )
    plt.show()
    plt.close(fig)

    # orginal
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    scan_og, meta_og = data_u.load_patient(all_eval_paths[idx])
    axes[0].hist(scan_og[..., 0].ravel(), bins=100, density=True)
    axes[0].set_title(all_eval_paths[idx])
    # corrected
    axes[1].hist(scan.ravel(), bins=100, density=True)
    axes[1].set_title(test_file)
    plt.tight_layout()
    plt.show()
    plt.close(fig)

In [ ]:
if False:
    # 3 annotated arteries, 4 annotations each artery
    markers = np.zeros((12, 3))
    annot_fnames = list(annotation_fnames[idx].glob("*.txt"))
    for i, artery_annotation_fname in enumerate(annot_fnames):
        print(artery_annotation_fname)
        markers[i * 4 : i * 4 + 4] = np.loadtxt(artery_annotation_fname)

    markers_image_coords = geom.world_to_image_coords(
        markers, meta["offset"], meta["spacing"]
    )
    markers_image_coords = markers_image_coords[markers_image_coords[:, 2].argsort()]

    fig, axes = plt.subplots(3, 4, figsize=(10, 7))
    for marker, ax in zip(markers_image_coords, axes.flat):
        ax.imshow(scan[..., marker[..., 2]].T, vmin=VMIN, vmax=VMAX, cmap="gray")
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.scatter(*marker[:2], c="red")
    fig.tight_layout()
    plt.show()
    plt.close(fig)

In [ ]:
if False:
    ctls = io_utils.load_centerlines(centerlines_scan_dir)[..., :3]
    print(ctls.shape)
    print(correction_fnames[idx])
    scan, meta = io_utils.load_sitk_image(correction_fnames[idx])
    ctls_img_coords = geom.world_to_image_coords(ctls, meta["offset"], meta["spacing"])
    print(ctls_img_coords.shape)

    for art, d in c.items():
        print(art)

        real_markers = np.loadtxt(list(Path(annotation_fnames[idx]).glob(f"{art}.txt"))[0])
        real_markers_img_c = geom.world_to_image_coords(
            real_markers, meta["offset"], meta["spacing"]
        )
        print("REAL")
        fig, axes = plt.subplots(1, 4, figsize=(15, 10))
        for p_img_c, ax in zip(real_markers_img_c, axes.flat):
            ax.imshow(scan[..., p_img_c[-1]].T, cmap="gray", vmin=VMIN, vmax=VMAX)
            ax.scatter(*p_img_c[:2], c="red")
            ax.set_xticks([]), ax.set_yticks([])
            ax.set_title(p_img_c)
        fig.tight_layout()
        plt.show()
        plt.close(fig)

        print("FOUND")
        fig, axes = plt.subplots(1, 4, figsize=(15, 10))
        for z_idx, ax, dist in zip(d["z_idx"], axes.flat, d["dist"]):
            p_img_c = ctls_img_coords[z_idx]
            ax.imshow(scan[..., p_img_c[-1]].T, cmap="gray", vmin=VMIN, vmax=VMAX)
            ax.scatter(*p_img_c[:2], c="red")
            ax.set_xticks([]), ax.set_yticks([])
            ax.set_title(str(tuple(p_img_c)) + " distance " + f"{dist:.2f} mm")
        fig.tight_layout()
        plt.show()
        plt.close(fig)

In [ ]:
results_ascoa, metrics_asoca = mrr.eval_model_marker_recall_rate(
    project.CENTERLINES_DIR / "ASOCA_Philips",
    project.ANNOTATIONS_DIR / "ASOCA_Philips",
    pd.read_excel(project.DATA_DIR / "ostia_final.xlsx"),
    annot_read_fn=mrr.read_ASOCA_annotations,
)

pprint(metrics_asoca)

Missing annonations for '/home/marco/data/annotations/ASOCA_Philips/ASOCA-035.ctl.anno'
{<ScanType.OPT: 0>: {'centerlines': 0.9150556064894169},
 <ScanType.LOW: -1>: {'centerlines': 0.934},
 <ScanType.HIGH: 1>: {'centerlines': 0.9101457219862044}}


In [ ]:
results, metrics = {}, {}
for centerlines_root_suffix in ["IDR_CADRADS", "IDR_CADRADS_3D-WC-small-rmsprop"]:
    print(centerlines_root_suffix)
    res, met = mrr.eval_model_marker_recall_rate(
        project.CENTERLINES_DIR / centerlines_root_suffix,
        project.ANNOTATIONS_DIR / "IDR_CADRADS",
        labels,
    )
    results[centerlines_root_suffix] = res
    metrics[centerlines_root_suffix] = met
    print()

pprint(metrics)

aggregated = {
    dataset: mrr.summarize_marker_recall_rate(d) for dataset, d in results.items()
}

pprint(aggregated)

IDR_CADRADS
'/home/marco/data/annotations/IDR_CADRADS/Contrast0472/LAD.txt' has only 3 annotations
Skip missing annotation '/home/marco/data/annotations/IDR_CADRADS/Contrast1052/RCA.txt'

IDR_CADRADS_3D-WC-small-rmsprop
'/home/marco/data/annotations/IDR_CADRADS/Contrast0472/LAD.txt' has only 3 annotations
Skip missing annotation '/home/marco/data/annotations/IDR_CADRADS/Contrast1052/RCA.txt'

{'IDR_CADRADS': {<ScanType.LOW: -1>: {'LAD': 0.7761194029850746,
                                      'LCX': 0.7794117647058824,
                                      'RCA': 0.90625},
                 <ScanType.HIGH: 1>: {'LAD': 0.9367088607594937,
                                      'LCX': 0.9018987341772152,
                                      'RCA': 0.9050632911392406}},
 'IDR_CADRADS_3D-WC-small-rmsprop': {<ScanType.LOW: -1>: {'LAD': 0.4925373134328358,
                                                          'LCX': 0.4852941176470588,
                                                    